# TLM FIFOS

In [1]:
from pyuvm import *

class Producer(uvm_component):
    def build_phase(self):
        self.pp = uvm_put_port("pp", self)
    
    def run_phase(self):
        self.raise_objection()
        for nn in range(5):
            self.pp.put(nn)
            print(f"Put {nn}", end=" ")
        self.drop_objection()
        
class Consumer(uvm_component):
    def build_phase(self):
        self.gp = uvm_get_port("gp", self)
    
    def run_phase(self):
        while True:
            nn = self.gp.get()
            print(f"Got {nn}", end=" ")

class FIFOTest(uvm_test):
    def build_phase(self):
        self.prod = Producer("prod", self)
        self.cons = Consumer("cons", self)
        self.fifo = uvm_tlm_fifo("fifo", self)
    
    def connect_phase(self):
        self.prod.pp.connect(self.fifo.put_export)
        self.cons.gp.connect(self.fifo.get_export)

uvm_root().run_test("FIFOTest")

Put 0 Got 0 Put 1 Got 1Put 2  Got 2 Put 3 Got 3 Put 4 Got 4 

## NonBlocking Functions in the UVM

In [1]:
from pyuvm import *
import time
import random
class Producer(uvm_component):
    def build_phase(self):
        self.pp = uvm_put_port("pp", self)
    
    def run_phase(self):
        self.raise_objection()
        for nn in range(5):
            success = False
            while not success:
                success = self.pp.try_put(nn)
                if success:
                    print(f"Put {nn}")
                else:
                    sleep_time = random.randint(1,10)/10
                    print(f"Failed to put {nn}. Sleep {sleep_time:.2f} seconds")
                    time.sleep(sleep_time)
        time.sleep(1)
        self.drop_objection()
        
class Consumer(uvm_component):
    def build_phase(self):
        self.gp = uvm_get_port("gp", self)
    
    def run_phase(self):
        while True:
            success = False
            while not success:
                success, nn = self.gp.try_get()
                if success:
                    print(f"Got {nn}")
                else:
                    sleep_time = random.randint(1,10)/10
                    print(f"Failed to get. Sleep {sleep_time:.1f} seconds")
                    time.sleep(sleep_time)
                    

class FIFOTest(uvm_test):
    def build_phase(self):
        self.prod = Producer("prod", self)
        self.cons = Consumer("cons", self)
        self.fifo = uvm_tlm_fifo("fifo", self)
    
    def connect_phase(self):
        self.prod.pp.connect(self.fifo.put_export)
        self.cons.gp.connect(self.fifo.get_export)

uvm_root().run_test("FIFOTest")

Put 0
Failed to put 1. Sleep 0.20 seconds
Got 0
Failed to get. Sleep 1.0 seconds
Put 1
Failed to put 2. Sleep 0.40 seconds
Failed to put 2. Sleep 0.40 seconds
Got 1Put 2
Failed to put 3. Sleep 0.30 seconds

Got 2
Failed to get. Sleep 0.7 seconds
Put 3
Failed to put 4. Sleep 0.90 seconds
Got 3
Failed to get. Sleep 0.6 seconds
Put 4
Got 4
Failed to get. Sleep 0.9 seconds
Failed to get. Sleep 0.9 seconds